<a href="https://colab.research.google.com/github/eyupdalan/BLM6114-hw3/blob/main/BLM6114_hw3_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install --upgrade fsspec==2025.3.0

In [18]:
!pip install -q transformers datasets peft trl accelerate bitsandbytes pandas openpyxl

### Veri Setini Hazırlama

In [19]:
from typing import Literal
from datasets import Dataset
import pandas as pd

gpt4o = "gpt4o cevabı"
deepseek = "deepseek cevabı"

def generate_dataframes(type: Literal[gpt4o, deepseek]):
  print(f"generate_dataframes:Generating {type} dataframe...")
  excel_data_frame = pd.read_excel("ogrenci_sorular_2025.xlsx", engine="openpyxl")
  result = excel_data_frame[["Soru", type]].copy()
  result.columns=["soru", "cevap"]

  return result

def generate_dataset(type):
  print(f"generate_dataset:Generating {type} dataset...")
  df = generate_dataframes(type)
  df["text"] = "### Soru:\n" + df["soru"].astype(str) + "\n\n### Cevap:\n" + df["cevap"].astype(str)
  return Dataset.from_pandas(df[["text"]])



In [20]:
ds = generate_dataset(gpt4o)

ds[:5]

generate_dataset:Generating gpt4o cevabı dataset...
generate_dataframes:Generating gpt4o cevabı dataframe...


{'text': ['### Soru:\n⦁\tŞu listedekilerin ortak özelliği nedir? Gözlük, Kalem, Laptop\n\n### Cevap:\nGözlük, kalem ve laptopun tek bir ortak özelliği, hepsinin **kişisel kullanım için tasarlanmış** olmalarıdır. Bu araçlar, bireylerin günlük yaşamlarında kişisel ihtiyaçlarını karşılamak için kullanılır.',
  '### Soru:\n"Adalet" kavramına benzer, ancak farklı toplumlarda veya zamanlarda farklı yorumlanan başka hangi soyut kavramlar vardır?\n\n### Cevap:\n"Adalet" kavramına benzer, ancak farklı toplumlarda veya zamanlarda farklı yorumlanan diğer soyut kavramlar arasında özgürlük, eşitlik, hak, ahlâk, güvenlik, sevgi, eşitlik ve dürüstlük yer alır. Özgürlük, bireylerin hareket etme ve düşünme özgürlüğüne dair farklı toplumlarda farklı şekillerde yorumlanabilir; bazı toplumlarda bireysel özgürlük ön planda iken, diğerlerinde toplumsal düzenin korunması öncelikli olabilir. Eşitlik, her bireyin aynı haklara sahip olması anlamına gelir, ancak bazı toplumlarda eşitlik sadece yasal açıdan değer

### Model Hazırlama

In [21]:
from typing import Literal
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from trl import SFTTrainer

medium = "ytu-ce-cosmos/turkish-gpt2-medium"
large = "ytu-ce-cosmos/turkish-gpt2-large"

def generate_model_and_tokenizer(model_id: Literal[medium, large]):
  print(f"generate_model_and_tokenizer:Generating {model_id}...")
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  model = AutoModelForCausalLM.from_pretrained(model_id)

  print(f"{model_id} generated...")
  return model, tokenizer

def fine_tune_and_save_model(model_id: Literal[medium, large], data_type: Literal["gpt4o", "deepseek"], dataset):
  print(f"fine_tune_and_save_model:Fine tuning {model_id}...")
  model, tokenizer = generate_model_and_tokenizer(model_id)

  lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules="all-linear",
    modules_to_save=["lm_head", "embed_token"],
    task_type="CAUSAL_LM",
    #r=8,
    #lora_alpha=32,
    #target_modules=["c_attn"],
    #lora_dropout=0.05,
    #bias="none",
    #task_type=TaskType.CAUSAL_LM,
  )

  training_args = TrainingArguments(
    output_dir=f"./{model_id}-{data_type}-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_dir=f"./{model_id}-{data_type}-logs",
    logging_steps=100,
    save_strategy="epoch",
    fp16=True,
    push_to_hub=False,
  )

  trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    #tokenizer=tokenizer,
    args=training_args,
    peft_config=lora_config,
    #dataset_text_field="text",
    #max_seq_length=512
  )

  trainer.train()
  print(f"{model_id}-{data_type} trained...")
  trainer.model.save_pretrained(f"./{model_id}-{data_type}-adapter")
  print(f"{model_id}-{data_type} model saved as ./{model_id}-{data_type}-adapter ...")
  trainer.tokenizer.save_pretrained(f"./{model_id}-{data_type}-adapter")
  print(f"{model_id}-{data_type} tokenizer saved as ./{model_id}-{data_type}-adapter ...")



### Kullanma

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel

def get_model_to_use(model_id: Literal[medium, large],data_type: Literal["gpt4o", "deepseek"]):
  print(f"get_model_to_use:Getting {model_id}...")
  adapter = f"./{model_id}-{data_type}-adapter"
  base_model = AutoModelForCausalLM.from_pretrained(model_id)
  tokenizer = AutoTokenizer.from_pretrained(adapter)
  model = PeftModel.from_pretrained(base_model, adapter)
  model.eval()
  return model, tokenizer

def generate_pipeline(model_id: Literal[medium, large], data_type: Literal["gpt4o", "deepseek"]):
  print(f"generate_pipeline:Generating {model_id} pipeline...")
  model, tokenizer = get_model_to_use(model_id, data_type)
  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
  return pipe

def run_prompt(question: str, pipe):
  print(f"run_prompt:Running {question}...")
  prompt = f"""
  ### Soru:
  {question}

  ### Cevap:
  """
  result = pipe(prompt, max_new_tokens=100, do_sample=True, temperature=0.7, top_p=0.95)
  return result[0]["generated_text"]



### Kontroller

##### Model-Adapter Kaydetme

In [23]:
gpt_dataset = generate_dataset(gpt4o)
deepseek_dataset = generate_dataset(deepseek)

fine_tune_and_save_model(medium,"gpt4o",gpt_dataset)
fine_tune_and_save_model(medium,"deepseek", deepseek_dataset)
#fine_tune_and_save_model(large,"gpt4o",gpt_dataset)
#fine_tune_and_save_model(large,"deepseek", deepseek_dataset)

generate_dataset:Generating gpt4o cevabı dataset...
generate_dataframes:Generating gpt4o cevabı dataframe...
generate_dataset:Generating deepseek cevabı dataset...
generate_dataframes:Generating deepseek cevabı dataframe...
fine_tune_and_save_model:Fine tuning ytu-ce-cosmos/turkish-gpt2-medium...
generate_model_and_tokenizer:Generating ytu-ce-cosmos/turkish-gpt2-medium...
ytu-ce-cosmos/turkish-gpt2-medium generated...


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/13891 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/13891 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/13891 [00:00<?, ? examples/s]

...

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,2.716400
200,2.530600
300,2.447600
400,2.423500
500,2.390100
600,2.366500
700,2.368400
800,2.307400
900,2.285900
1000,2.191500


ytu-ce-cosmos/turkish-gpt2-medium-deepseek trained...


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


ytu-ce-cosmos/turkish-gpt2-medium-deepseek model saved as ./ytu-ce-cosmos/turkish-gpt2-medium-deepseek-adapter ...
ytu-ce-cosmos/turkish-gpt2-medium-deepseek tokenizer saved as ./ytu-ce-cosmos/turkish-gpt2-medium-deepseek-adapter ...


In [28]:
pipe_gpt = generate_pipeline(medium, "gpt4o")
pipe_deepseek = generate_pipeline(medium, "deepseek")

result_gpt = run_prompt("zebra mı daha hızlıdır at mı?", pipe_gpt)
print("result_gpt_medium",result_gpt)

result_deepseek = run_prompt("zebra mı daha hızlıdır at mı?", pipe_deepseek)
print("result_deepseek_medium",result_deepseek)

generate_pipeline:Generating ytu-ce-cosmos/turkish-gpt2-medium pipeline...
get_model_to_use:Getting ytu-ce-cosmos/turkish-gpt2-medium...


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

generate_pipeline:Generating ytu-ce-cosmos/turkish-gpt2-medium pipeline...
get_model_to_use:Getting ytu-ce-cosmos/turkish-gpt2-medium...


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

run_prompt:Running zebra mı daha hızlıdır at mı?...
result_gpt_medium 
  ### Soru:
  zebra mı daha hızlıdır at mı?

  ### Cevap:
   - Zebra daha hızlıdır, ancak at daha yavaştır.
run_prompt:Running zebra mı daha hızlıdır at mı?...
result_deepseek_medium 
  ### Soru:
  zebra mı daha hızlıdır at mı?

  ### Cevap:
   Zebra, hızlı olduğu için at daha hızlıdır.


In [30]:
import pandas as pd

# Load questions from the file
with open("yeni_sorular_50_adet.txt", "r", encoding="utf-8") as f:
    questions = f.readlines()
    questions = [q.strip() for q in questions if q.strip()]  # Remove empty lines

# Function to run prompt and get responses
def get_responses(question):
  result_gpt_medium = run_prompt(question, pipe_gpt)
  result_deepseek_medium = run_prompt(question, pipe_deepseek)
  return result_gpt_medium, result_deepseek_medium

# Generate and print responses for all questions
all_responses = []
for question in questions:
  result_gpt_medium, result_deepseek_medium = get_responses(question)
  print(f"Question: {question}")
  print(f"GPT Medium Response: {result_gpt_medium}")
  print(f"Deepseek Medium Response: {result_deepseek_medium}")
  print("-" * 50)

  all_responses.append({
      "question": question,
      "gpt_response_medium": result_gpt_medium,
      "deepseek_response_medium": result_deepseek_medium,
  })

# Optionally save responses to a file
df = pd.DataFrame(all_responses)
df.to_excel("responses.xlsx", index=False)
print("Responses saved to responses.xlsx")

run_prompt:Running Şu nesnelerin ortak özelliği nedir? Kalem, silgi, cetvel...
run_prompt:Running Şu nesnelerin ortak özelliği nedir? Kalem, silgi, cetvel...
Question: Şu nesnelerin ortak özelliği nedir? Kalem, silgi, cetvel
GPT Medium Response: 
  ### Soru:
  Şu nesnelerin ortak özelliği nedir? Kalem, silgi, cetvel

  ### Cevap:
   Kalem, silgi, cetvel, not defteri, hesap makinesi ve diğer kırtasiye ürünleri hepsi kırtasiye ürünlerdir.
Deepseek Medium Response: 
  ### Soru:
  Şu nesnelerin ortak özelliği nedir? Kalem, silgi, cetvel

  ### Cevap:
   - Kalem ve silgi, yazı yazmak için kullanılan nesnelerdir.  
- Silgi ve cetvel ise nesneleri süslemek veya göstermek için kullanılır.  

🔍 **Ek Bilgi:**  
- Kalem ve silgi, yazı yazmak için kullanılır, ancak süs amaçlı kullanılmaz.  

🔍 **Fark:**  
- Kalem ve silgi, yazı yazmak için kullanılmaz.  
- Kalem ve silgi, yazı yazmak için kullanılan nesnelerdir.  

📌 *Not
--------------------------------------------------
run_prompt:Running Şu lis